In [1]:
pip install simpletransformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl.metadata (2.1 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 1.22.4 which is incompatible.


Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [2]:
pip install wandb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
dataset_path = "/content/train.csv"
df = pd.read_csv(dataset_path)

# Split dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Convert data to required format for Simple Transformers
train_data = []
for index, row in train_df.iterrows():
    train_data.append({"qas_id": str(index), "question": row["Question"], "context": row["Answer"], "answers": {"text": [row["Answer"]], "answer_start": [0]}})

test_data = []
for index, row in test_df.iterrows():
    test_data.append({"qas_id": str(index), "question": row["Question"], "context": row["Answer"], "answers": {"text": [row["Answer"]], "answer_start": [0]}})

df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [8]:
from simpletransformers.question_answering import QuestionAnsweringModel

bert_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "train_batch_size": 8,
    "learning_rate": 3e-5,
    "doc_stride": 128,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "wandb_project": "medquad-bert",
    "use_cuda": False  # Set use_cuda to False
}

bert_model = QuestionAnsweringModel("bert", "bert-base-uncased", args=bert_args)
bert_model.train_model(train_data, eval_data=test_data)

# Fine-tune MobileBERT
mobilebert_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "train_batch_size": 8,
    "learning_rate": 3e-5,
    "doc_stride": 128,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "wandb_project": "medquad-mobilebert",
    "use_cuda": False  # Set use_cuda to False
}

mobilebert_model = QuestionAnsweringModel("mobilebert", "google/mobilebert-uncased", args=mobilebert_args)
mobilebert_model.train_model(train_data, eval_data=test_data)

# Fine-tune RoBERTa
roberta_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "train_batch_size": 8,
    "learning_rate": 3e-5,
    "doc_stride": 128,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "wandb_project": "medquad-roberta",
    "use_cuda": False  # Set use_cuda to False
}

roberta_model = QuestionAnsweringModel("roberta", "roberta-base", args=roberta_args)
roberta_model.train_model(train_data, eval_data=test_data)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: 'use_cuda' set to True when cuda is unavailable. Make sure CUDA is available or set use_cuda=False.

In [9]:
result, texts = bert_model.eval_model(test_data)

NameError: name 'bert_model' is not defined

In [10]:
# Define hyperparameters to tune
hyperparameters = {
    "dropout_rate": [0.3, 0.5, 0.7],
    "num_hidden_layers": [6, 8, 12],
    "learning_rate": [1e-5, 2e-5, 3e-5]
}

best_f1_score = 0
best_hyperparameters = None

# Loop through all combinations of hyperparameters
for dropout_rate in hyperparameters["dropout_rate"]:
    for num_hidden_layers in hyperparameters["num_hidden_layers"]:
        for learning_rate in hyperparameters["learning_rate"]:
            # Define model with current hyperparameters
            model_args = {
                "dropout_rate": dropout_rate,
                "num_hidden_layers": num_hidden_layers,
                "learning_rate": learning_rate,
                # Add other relevant hyperparameters here
            }

            # Fine-tune model with current hyperparameters
            model = QuestionAnsweringModel("bert", "bert-base-uncased", args=model_args)
            model.train_model(train_data, eval_data=test_data)

            # Evaluate model on validation set
            result, _ = model.eval_model(test_data)
            f1_score = result["f1"]

            # Check if current hyperparameters yield better performance
            if f1_score > best_f1_score:
                best_f1_score = f1_score
                best_hyperparameters = model_args

# Print best hyperparameters and corresponding F1 score
print("Best Hyperparameters:", best_hyperparameters)
print("Best F1 Score:", best_f1_score)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: 'use_cuda' set to True when cuda is unavailable. Make sure CUDA is available or set use_cuda=False.

In [15]:
import wandb

wandb.login()

wandb.init(project="medquad-bert")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [16]:
for text in texts[:5]:
    print("Question:", text["question"])
    print("Actual Answer:", text["context"])
    print("Predicted Answer:", text["answer"])
    print()


NameError: name 'texts' is not defined

In [17]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge

# Calculate BLEU Score
def calculate_bleu_scores(actual, predicted):
    references = [[actual]]
    candidates = [predicted]
    return corpus_bleu(references, candidates)

# Calculate Rouge Score
def calculate_rouge_scores(actual, predicted):
    rouge = Rouge()
    scores = rouge.get_scores(predicted, actual)
    return scores

# Usage
bleu_score = calculate_bleu_scores(actual, predicted)
rouge_scores = calculate_rouge_scores(actual, predicted)


ModuleNotFoundError: No module named 'rouge'